# 1. 라이브러리 import

## 1) 모델 관련

In [ ]:
import os

# 음성 데이터 처리
import librosa
import numpy as np
import pandas as pd

# 데이터 시각화
from matplotlib import pyplot as plt

# 모델 관련
import sklearn
from sklearn import preprocessing # AttributeError: module 'sklearn' has no attribute 'preprocessing'
import tensorflow as tf

## 2) 음성 강세 분석 모델 사용을 위한 라이브러리

In [ ]:
import import_ipynb
import voice_strength

## 3) whisper 사용을 위한 라이브러리

In [ ]:
import whisper
import torch
import threading
import queue

## 4) 플라스크 서버 구축을 위한 라이브러리

In [ ]:
import tempfile
import logging
from flask import Flask, request, jsonify
import requests
from flask_restx import Api, Resource, fields, marshal
import signal
from datetime import datetime

# 2. GPU 사용 확인

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.config.list_physical_devices('GPU'))

# 3. 이것 저것 설정

In [ ]:
torch.cuda.empty_cache()

In [ ]:
task_queue = queue.Queue()
results = {}

num_worker_threads = 1

S_presigned_url = 'objectUrl'
S_note_id = 'noteId'
C_process_id = 'processId'
backEnd_URL = "http://REMOVED:8080/voice/sttResult"

shutdown_flag = threading.Event()

# 4. 서버 구축

## 1) 로그 설정

In [ ]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

## 2) 스레드 설정

In [ ]:
def worker(thread_id):
    logger.info(f"Worker {thread_id} starting, loading Whisper model...")
    model = whisper.load_model("large-v3")  # Adjust the model size as needed
    logger.info(f"Worker {thread_id} loaded Whisper model.")

    while not shutdown_flag.is_set():  # Check for shutdown signal
        try:
            data = task_queue.get(timeout=1)  # 딕셔너리에서 값 꺼내기
            note_id = data["note_id"]
            temp_audio_path = data["temp_audio_path"]
            process_id = data["process_id"]
            
        except queue.Empty:
            continue
        
        if note_id is None:
            break

        logger.info(f"Worker {thread_id} processing task {note_id}")
        
        try:
            result = model.transcribe(temp_audio_path)  # temp_audio_path를 사용해야 합니다
            send_result_to_spring(note_id, process_id, result['segments'])
        except Exception as e:
            logger.error(f"Worker {thread_id} encountered an error: {str(e)}")
            # results[note_id] = {"status": "failed", "error": str(e)}
        finally:
            os.remove(temp_audio_path)  # temp_audio_path로 변경
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

        task_queue.task_done()

In [ ]:
threads = []
for i in range(num_worker_threads):
    t = threading.Thread(target=worker, args=(i,))
    t.start()
    threads.append(t)

## 2) 라우터 및 swagger 설정

In [ ]:
app = Flask(__name__)

# swagger (http://127.0.0.1:5000/api-docs)
api = Api(
    app,
    version='1.0',
    title='발화 강세 분석 AI를 위한 API 문서',
    description='Swagger 문서',
    doc="/api-docs"
)

voice_api = api.namespace(name='voice', description='발화 분석 관련 API')
test_api = api.namespace('voice_stt', description='STT 기능 관련 API')
index_api = api.namespace('hello', description='index API')

## 3) dto 정의

In [ ]:
analysis_request_model = api.model('request', {
    'noteId': fields.Integer(readOnly=True, required=True, description='노트 id'), 
    'objectUrl': fields.String(required=True, description='녹음본이 저장된 위치'),
    'processId': fields.String(required=True, description='각 요청에 대한 UUID'),
})

analysis_response_model = api.model('response', {
    'noteId': fields.String(readOnly=True, required=True, description='노트 id'),
    'processId': fields.String(required=True, description='각 요청에 대한 UUID'),
    'status': fields.String(required=True, description='처리 상태'),
})

analysis_response_data = {
    'noteId': 0,
    'processId': '',
    'status': '',
}

base_response = {
    'code': 200,
    'message': '',
    'data': None
}

# 요청 모델 정의
stt_request_model = api.model('STTRequest', {
    S_note_id: fields.String(required=True, description='Note ID'),
    S_presigned_url: fields.String(required=True, description='Pre-signed URL for audio file'),
    C_process_id: fields.String(required=True, description='Process ID')
})

# 응답 모델 정의
stt_response_model = api.model('STTResponse', {
    S_note_id: fields.String(description='Note ID'),
    C_process_id: fields.String(required=True, description='Process ID'),
    'status': fields.String(description='Processing status')
})

## 4) 컨트롤러

In [ ]:
@voice_api.route('/')
class Index(Resource):
    def get(self):
        base_response['code'] = 200
        base_response['message'] = 'hello'
        base_response['data'] = '안녕하'

        return jsonify(base_response)

In [ ]:
@test_api.route('/stt')
class STT(Resource):

    @api.expect(stt_request_model)  # 요청 모델을 Swagger에 추가
    @api.marshal_with(stt_response_model)  # 응답 모델을 Swagger에 추가
    def post(self):
        client_ip = request.remote_addr
        logger.info("check STT post method's connector: "+client_ip)


        logger.info("Received STT request")

        process_id = request.json.get(C_process_id)
        audio_uri = request.json.get(S_presigned_url)
        note_id = request.json.get(S_note_id)

        logger.info(f"들어온 URL: {audio_uri}");
        logger.info(f"들어온 URL: {process_id}");

        logger.info(f"들어온 note_id: {note_id}");

        if not request.json.get(S_presigned_url):
            logger.warning("No audio file provided in the request")
            return {'error': 'No audio file provided'}, 400

        if audio_uri:
            try:
                logger.info(f"Downloading audio file from URI: {audio_uri}")
                response = requests.get(audio_uri)
                response.raise_for_status()

                with tempfile.NamedTemporaryFile(delete=False) as temp_audio:
                    temp_audio.write(response.content)
                    temp_audio_path = temp_audio.name

                logger.info(f"Audio file downloaded and saved to {temp_audio_path}")

                data = {
                    "note_id": note_id,
                    "temp_audio_path": temp_audio_path,
                    "process_id": process_id
                }
                task_queue.put(data)

                logger.info(f"Task {note_id} added to the queue")

                return {S_note_id: note_id, "processId": process_id, "status": "processing"}, 202

            except requests.exceptions.RequestException as e:
                logger.error(f"Failed to download audio file: {str(e)}")
                return {'error': 'Failed to download audio file'}, 400

In [ ]:
@voice_api.route('/analysis')
class Analysis(Resource):
    @voice_api.expect(analysis_request_model)  # 요청 모델을 Swagger에 추가
    # @api.marshal_with(analysis_response_model)  # 응답 모델을 Swagger에 추가

    def post(self):
        try:
            req = marshal(request.get_json(force=True), analysis_request_model)
            print(f"{datetime.now().time()} {req}")
            
            # 유효 범위 처리
            if req['noteId'] < 0:
                return


            audio_url = req['objectUrl']
            
            if audio_url.strip() != '':
                try:
                    print(f"{datetime.now().time()} Downloading audio file from URI: {audio_url}")
                    logger.info(f"{datetime.now().time()} Downloading audio file from URI: {audio_url}")
                    
                    response = requests.get(audio_url) # .wav 파일 다운로드
                    response.raise_for_status()  # 오류 발생 시 예외 처리
                    
                    # 오디오 데이터를 저장할 임시 파일 생성
                    with tempfile.NamedTemporaryFile(delete=False) as temp_audio:
                        temp_audio.write(response.content)  # 다운로드한 파일 내용을 임시 파일에 저장
                        temp_audio_path = temp_audio.name  # 임시 파일 경로 저장

                    print(f"{datetime.now().time()} Audio file downloaded and saved to {temp_audio_path}")
                    logger.info(f"{datetime.now().time()}  Audio file downloaded and saved to {temp_audio_path}")



                    # time_list = send_analysis_result(req['noteId'], req['processId'], temp_audio_path)
                    # analysis_response_data['noteId'] = req['noteId']
                    # analysis_response_data['processId'] = req['processId']
                    # analysis_response_data['status'] = time_list
                    
                    # 비동기로 음성 분석 실행
                    task_thread = threading.Thread(target=send_analysis_result, args=(req['noteId'], req['processId'], temp_audio_path,))
                    task_thread.start()
                    
                    # 작업 ID와 상태 반환
                    analysis_response_data['noteId'] = req['noteId']
                    analysis_response_data['processId'] = req['processId']
                    analysis_response_data['status'] = 'processing'
                    

                    return analysis_response_data, 202
                    
                except requests.exceptions.RequestException as e:
                    logger.error(f"Failed to download audio file: {str(e)}")
                    return jsonify({'error': 'Failed to download audio file'}), 400

        except ValueError as e:
            base_response['code'] = 400
            base_response['message'] = 'error: ' + str(e)
            return jsonify(base_response)
        
        except Exception as e:
            base_response['code'] = 500
            base_response['message'] = 'error: ' + str(e)
            return jsonify(base_response)

## 5) 기타 메서드

In [ ]:
# 비동기 작업
def send_analysis_result(noteId, processId, audio_path):
    second_list = voice_strength.get_abnormal_interval('', [audio_path])

    data = {
        'id': int(noteId),
        'process_id': str(processId),
        'anomaly_time': second_list
    }
    
    # 스프링의 /voice/analysisResult api 호출
    try:
        response = requests.post("http://REMOVED:8080/voice/analysisResult", json=data)
        print(f"{datetime.now().time()} request: {data}")
        print(f"{datetime.now().time()} responst: {response}")
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"{datetime.now().time()} 결과 전송 오류: {e}")

## 6) 에러 핸들링

In [ ]:
@app.errorhandler(405)
def internal_error(error):
    logger.error(f"Method Not Allowed: {str(error)}")

    base_response['code'] = 405
    base_response['message'] = 'Method Not Allowed'

    return jsonify(base_response)

In [ ]:
@app.errorhandler(500)
def internal_error(error):
    logger.error(f"Internal Server Error: {str(error)}")
    return {"error": "Internal Server Error"}, 500

## 7) main

In [ ]:
if __name__ == '__main__':
    try:
        #public_url = ngrok.connect(port).public_url
        #print(public_url)
        app.run(port=4998)

    finally:
        torch.cuda.empty_cache()
        handle_shutdown(None, None)
        for _ in range(num_worker_threads):
            task_queue.put((None, None))
        for t in threads:
            t.join()